In [75]:
import os
import numpy as np
import cv2
data_path = r"E:\data_share_ths\dataset\UCF-101\images\test"

In [76]:

image_labels  = {}
images_folder_counts = {}


for folder_name in os.listdir(data_path):
    class_name = folder_name.split("_")[1].lower()
    image_paths = []
    images_folder_counts[folder_name] = (len(os.listdir(data_path+"\\"+folder_name)))
    for filename in os.listdir(data_path+"\\"+folder_name):
        image_paths.append(data_path+"\\"+folder_name+"\\"+filename)
    image_labels[class_name] = image_paths

print(image_labels.keys())
print(len(image_labels['applyeyemakeup']))
print(len(image_labels['applylipstick']))
print(len(image_labels['archery']))



dict_keys(['applyeyemakeup', 'applylipstick', 'archery'])
115
309
158


In [77]:
img_interval = 5
batch_size = 8

In [78]:
### Add label for black image 
real_label = []
real_images =[]
for className,image_paths in image_labels.items():
    temp_img_path = image_paths
    while len(temp_img_path)%img_interval != 0:
        temp_img_path.append("-")
        print(len(temp_img_path))

    image_labels[className] = temp_img_path
    


310
159
160


In [90]:
print(image_labels.keys())
print(len(image_labels['applyeyemakeup']))
print(len(image_labels['applylipstick']))
print(len(image_labels['archery']))


dict_keys(['applyeyemakeup', 'applylipstick', 'archery'])
115
310
160


In [101]:
# Create X_train,Y_train, X_test,Y_test
test_amount = 50
X_train,Y_train= [],[]
X_test,Y_test = [],[]

for className,image_paths in image_labels.items():
    for i in range(len(image_paths)):
        
        if i < len(image_paths) - test_amount:
            y_img_path = image_paths[i]
            X_train.append(y_img_path)
            Y_train.append([className])

        else:
            X_test.append(y_img_path)
            Y_test.append([className])


print("X_train : ",len(X_train))
print("Y_train : ",len(Y_train))

print("X_test : ",len(X_test))
print("Y_test : ",len(Y_test))




X_train :  435
Y_train :  435
X_test :  150
Y_test :  150


### Custom data generator

In [102]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
CHANNEL = 3

BATCH_SIZE = 16
OUTPUT_CHANNELS = 3
EPOCHS = 5

In [100]:
import tensorflow as tf

In [109]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, X_col, Y_col,
                 batch_size,
                 input_size=(224, 224, 3)):
        self.X_col =X_col
        self.Y_col =Y_col
        self.batch_size = batch_size
        self.n = len(self.X_col)

    def read_img(self,path):
        img = cv2.imread(path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))
        
        #convert to tf and normalize
        tf_img = tf.cast(img, tf.float32)
        tf_img = (tf_img/127.5)-1
        return tf_img
    
    def create_emptyImage(self):
        img= np.zeros((IMG_WIDTH,IMG_HEIGHT,CHANNEL),dtype=int)
        tf_img = tf.cast(img, tf.float32)
        tf_img = (tf_img/127.5)-1
        return tf_img



    def __get_input(self, path):
    
        cur_img = None
        if path =="-":
            cur_img = self.create_emptyImage()
        else:
            cur_img = self.read_img(path)

        return cur_img
    

    
    def __get_output(self, label):
        return label
    
    def __get_data(self, x_batches, y_batches):
        # Generates data containing batch_size samples
        X_batchs = np.asarray([self.__get_input(x) for x in x_batches])
        y_batches = np.asarray(self.__get_output(y_batches))
        return X_batchs,y_batches
    
    def __getitem__(self, index):
        
        X_batches = self.X_col[index*self.batch_size : (index+1)*self.batch_size]
        Y_batches = self.Y_col[index*self.batch_size : (index+1)*self.batch_size]
        X,y= self.__get_data(X_batches,Y_batches)       
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size
        

In [110]:
test_gen = CustomDataGen(X_col=X_train,Y_col=Y_train,batch_size=batch_size)

In [112]:
for img,label in test_gen:
    print(label)
    print(len(img))
    break

[['applyeyemakeup']
 ['applyeyemakeup']
 ['applyeyemakeup']
 ['applyeyemakeup']
 ['applyeyemakeup']
 ['applyeyemakeup']
 ['applyeyemakeup']
 ['applyeyemakeup']]
8
